# Примеры заполнения таблицы

## Заполенение средствами psycopg2

Способ используется при небольших объемах данных. 

In [ ]:
import psycopg2
import os

In [ ]:
# подключение к базе данных
HOST=os.environ['GREENPLUM_HOST']
PORT=os.environ['GREENPLUM_PORT']
DATABASE=os.environ['GREENPLUM_DATABASE']
SCHEMA = os.environ['GREENPLUM_SCHEMA']
LOGIN=os.environ['GREENPLUM_LOGIN']
PASSWORD=os.environ['GREENPLUM_PASSWORD']

connection = psycopg2.connect( host=HOST, port=PORT, database=DATABASE, user=LOGIN, password=PASSWORD)

In [ ]:
# имя таблицы созданной в предыдущих примерах
tableName = 'table1'

In [ ]:
values= [
        (1,1.0,'1'),
        (2,2.0,'2'),
        (3,3.0,'3')
    ]

In [ ]:
sql = f"""
    insert into {SCHEMA}.{tableName} (c1, c2, c3) values (%s, %s, %s)
"""
cur = connection.cursor()
cur.executemany(sql,values)
connection.commit()
cur.close()

In [ ]:
connection.close()

In [ ]:
# проверка записанных данных
connection = psycopg2.connect( host=HOST, port=PORT, database=DATABASE, user=LOGIN, password=PASSWORD)
sql = f"""
     select * FROM {SCHEMA}.{tableName}
"""
cur = connection.cursor()
cur.execute(sql)
data = cur.fetchall()
print(data)
connection.commit()
cur.close()
connection.close()

## Заполенение средствами spark

Используется на больших объемах данных.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, LongType, StringType, IntegerType, DateType, TimestampType, FloatType
from pyspark.sql.functions import col, cast, date_trunc, sum, dayofweek, hour, dayofmonth, lit


In [ ]:
# запуск spark
spark = SparkSession.builder.appName('fill').getOrCreate()

In [ ]:
jdbcURLGreenplum = f'jdbc:postgresql://{HOST}:{PORT}/{DATABASE}?user={LOGIN}&password={PASSWORD}'


In [ ]:
# формирование dataframe
values= [
        (4,4.0,'4'),
        (5,5.0,'5'),
        (6,6.0,'6')
    ]

schemaData = StructType([ \
    StructField("c1",IntegerType(),True), 
    StructField("c2",FloatType(),True), 
    StructField("c3",StringType(),True)
  ])

df = spark.createDataFrame(values,schema=schemaData)

In [ ]:
df.show(3,False)

In [ ]:
# запись в таблицу
df.write.format("jdbc").mode("append") \
              .option('url', jdbcURLGreenplum) \
              .option('driver', 'org.postgresql.Driver') \
              .option("dbtable",SCHEMA+'.'+tableName).save()

In [ ]:
# чтение из таблицы
query=f'''
SELECT  *
FROM {SCHEMA}.{tableName}
'''
df1 = spark.read.format("jdbc") \
              .option('url', jdbcURLGreenplum) \
              .option('driver', 'org.postgresql.Driver') \
              .option('query',query).load()
df1.show()

In [ ]:
spark.stop()